# Pokemon

## Synopsis

### Background

This dataset is interesting because there is an abundance of classification variables which have unknown effects of the label. Because these classification variables could have different effects on the result depending on which is against which, whether there are two or just one, etc, the number of variables boolean variables needed to encapsulate all the possibible combinations (i.e. Flying Grass vs Flying Fire, Flying Grass vs Grass Poison, etc) would have been excessive. I therefore decided to just have booleans for all the possible type (there were 18) for each pokemon and apply a neural network. The data is split into two seperate files, a test and train file. The test file has no labels, however, as this dataset was pulled from a competition on Kaggle. I therefore treated the train dataset as the complete dataset and partitioned it into test and train subsets.

### Results

The only model trained on this dataset was a 2 layer neural network. The resulting accuracy for the training set was 98.8% while the test set came in at 95.3%, meaning the model generalized quite well.

In [36]:
#!/usr/bin/env python3

import h5py as h5py
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
%matplotlib inline

file_path = "../data/pokemon/"

file_combats = "combats.csv"
file_pokemon = "pokemon.csv"
file_tests = "tests.csv"



In [3]:
combats = pd.read_csv(file_path + file_combats)
pokemon = pd.read_csv(file_path + file_pokemon)

In [4]:
combats.head(20)

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151
5,657,752,657
6,192,134,134
7,73,545,545
8,220,763,763
9,302,31,31


In [5]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
#             800 non-null int64
Name          799 non-null object
Type 1        800 non-null object
Type 2        414 non-null object
HP            800 non-null int64
Attack        800 non-null int64
Defense       800 non-null int64
Sp. Atk       800 non-null int64
Sp. Def       800 non-null int64
Speed         800 non-null int64
Generation    800 non-null int64
Legendary     800 non-null bool
dtypes: bool(1), int64(8), object(3)
memory usage: 69.6+ KB


In [6]:
pokemon.head(10)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
5,6,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,False
6,7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False
7,8,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False
8,9,Mega Charizard Y,Fire,Flying,78,104,78,159,115,100,1,False
9,10,Squirtle,Water,NaN,44,48,65,50,64,43,1,False


In [7]:
pokemon.isnull().any()

#             False
Name           True
Type 1        False
Type 2         True
HP            False
Attack        False
Defense       False
Sp. Atk       False
Sp. Def       False
Speed         False
Generation    False
Legendary     False
dtype: bool

In [10]:
pokemon.describe()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.0000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,400.5000,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,231.0844,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.0000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,200.7500,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,400.5000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,600.2500,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,800.0000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [9]:
combats.isnull().any()

First_pokemon     False
Second_pokemon    False
Winner            False
dtype: bool

In [12]:
combats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
First_pokemon     50000 non-null int64
Second_pokemon    50000 non-null int64
Winner            50000 non-null int64
dtypes: int64(3)
memory usage: 1.1 MB


### Transformations

The functions below are the transformations that I applied to the dataset. The first two simply create binaries for the Types and Generations while the last is a quick normalization function. These functions are applied to the reference dataset containing the attributes of the pokemon.

Note that though null values were present in the dataset, they were contained to the second type column, and were present only if the character had just one type. For this reason the nulls were not filled or removed, as they would only be present in the final dataset. The combat dataset contained no nulls.

The last function creates the final form dataset on which the neural network will be trained. It takes the combat dataset and substitutes the id of the combatants with the attributes from the reference dataset. The order of the combatants is unchanged (i.e. the attributes of the pokemon on the left are placed on the left). The label variable is simply a boolean for if the pokemon on the left wins or not. While the function is straightforward, it is computationally expensive because there are 50,000 events and for each two lookups on the reference dataset (which itself has 800 elements) is required. The operation could have been improved with a mapping function and multithreading, but the implementation really didn't seem worth it, as the time for the function to complete is under 15 minutes.

In [13]:
def create_typeBinaries(data,types,cols):
    for ptype in types:
        data[str("binary_"+ptype)] = [1 if (ptype in [data[str(y)][i] for y in cols]) else 0 for i in range(len(data[cols[0]]))]
    return(data)

def create_generationBinaries(data,columns):
    for column in columns:
        for unique_value in data[column].unique():
            data[str(str(column)+str(unique_value))] = [1 if data[column][i] == unique_value else 0 for i in range(len(data[column]))]
    return(data)

def normalize(mydata,columns):
    for i in columns:
        temp_max = max(mydata[i])
        temp_min = min(mydata[i])
        temp = [(x-temp_min)/temp_max for x in mydata[i]]
        mydata[i] = temp
    return(mydata)


######### CODER BEWARE ##########
# Running this function on the  #
# dataset takes quite a while   #
# and will make your computer   #
# quite loud for a bit...       #
#       ...hashtag inefficient  #
#################################

def getFinalForm(combats,pokemon,binaries,num_columns):
    final = pd.DataFrame()
    final["Y"] = [1 if combats["First_pokemon"][x] == combats["Winner"][x] else 0 for x in range(len(combats))]
    final["first ID"] = combats["First_pokemon"]
    final["second ID"] = combats["Second_pokemon"]
    
    # for the winner, add the relevant binaries (Generation, Legendary, Types)
    for biny in binaries:
        final[str("first_" + biny)] = [int(pokemon.loc[pokemon["#"] == pok_id,biny]) for pok_id in final["first ID"]]
        final[str("second_" + biny)] = [int(pokemon.loc[pokemon["#"] == pok_id,biny]) for pok_id in final["second ID"]]
    
    for num_col in num_columns:
        #final[str("winner"+num_col+"_adv")] = [int(pokemon.loc[pokemon["#"] == final["winner ID"][i],num_col]) - int(pokemon.loc[pokemon["#"] == final["loser ID"][i],num_col]) for i in range(len(final["winner ID"]))]
        final[str("first_"+num_col)] = [(pokemon.loc[pokemon["#"] == final["first ID"][i],num_col]).item() for i in range(len(final["first ID"]))]
        final[str("second_"+num_col)] = [(pokemon.loc[pokemon["#"] == final["second ID"][i],num_col]).item() for i in range(len(final["second ID"]))]
        
    return(final)



In [14]:
# Get all the unique types in the dataset

types_list = []

for i in pokemon["Type 1"].unique():
    types_list.append(i)
for i in pokemon["Type 2"].unique():
    if i not in types_list:
        types_list.append(i)
        
types_list = [x for x in types_list if type(x) == str]
for i in types_list:
    print("binary_"+i)
    
# initialize columns and create binaries
    
num_columns = ["HP","Attack","Defense","Sp. Atk","Sp. Def","Speed"]
pokemon = create_typeBinaries(pokemon,types_list,["Type 1","Type 2"])
pokemon = create_generationBinaries(pokemon,["Generation"])
pokemon["Legendary"] = [1 if x == True else 0 for x in pokemon["Legendary"]]
binaries = [x for x in pokemon.columns if "binary" in str(x) or "Generation" in str(x)] + ["Legendary"]
pokemon = normalize(pokemon,["HP","Attack","Defense","Sp. Atk","Sp. Def","Speed"])


# Running the function below will take some time...
final_data = getFinalForm(combats,pokemon,binaries,num_columns)
final_data.to_csv("data_final_form.csv",sep=",",header=True)

binary_Grass
binary_Fire
binary_Water
binary_Bug
binary_Normal
binary_Poison
binary_Electric
binary_Ground
binary_Fairy
binary_Fighting
binary_Psychic
binary_Rock
binary_Ghost
binary_Ice
binary_Dragon
binary_Dark
binary_Steel
binary_Flying


In [10]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary', 'binary_Grass',
       'binary_Fire', 'binary_Water', 'binary_Bug', 'binary_Normal',
       'binary_Poison', 'binary_Electric', 'binary_Ground', 'binary_Fairy',
       'binary_Fighting', 'binary_Psychic', 'binary_Rock', 'binary_Ghost',
       'binary_Ice', 'binary_Dragon', 'binary_Dark', 'binary_Steel',
       'binary_Flying'],
      dtype='object')

In [15]:
# training the model

classifier = Sequential()

classifier.add(Dense(input_dim=62,activation='relu',init='uniform',output_dim=62))
classifier.add(Dense(input_dim=62,activation='relu',init='uniform',output_dim=62))
classifier.add(Dense(input_dim=62,activation='sigmoid',init='uniform',output_dim=1))

/Users/jakubkasan/coding/data-science/pokemon/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/jakubkasan/coding/data-science/pokemon/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=62, activation="relu", units=62, kernel_initializer="uniform")`
  if __name__ == '__main__':
/Users/jakubkasan/coding/data-science/pokemon/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=62, activation="relu", units=62, kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.
/Users/jakubkasan/coding/data-science/pokemon/venv/lib/pytho

In [16]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
y = final_data["Y"]
x = final_data.drop(["Y","first ID","second ID","first_Generation","second_Generation"],axis=1)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [20]:
classifier.fit(x_train,y_train,batch_size=5000,nb_epoch=1000)

/Users/jakubkasan/coding/data-science/pokemon/venv/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1000
35000/35000 [==============================] - 0s 13us/step - loss: 0.6925 - acc: 0.5173
Epoch 2/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.6897 - acc: 0.5361
Epoch 3/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.6821 - acc: 0.5893
Epoch 4/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.6664 - acc: 0.6473
Epoch 5/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.6408 - acc: 0.6857
Epoch 6/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.6068 - acc: 0.7076
Epoch 7/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.5714 - acc: 0.7217
Epoch 8/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.5410 - acc: 0.7344
Epoch 9/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.5183 - acc: 0.7502
Epoch 10/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.4996 - 

35000/35000 [==============================] - 0s 2us/step - loss: 0.2773 - acc: 0.9215
Epoch 82/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2761 - acc: 0.9218
Epoch 83/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2752 - acc: 0.9228
Epoch 84/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2743 - acc: 0.9215
Epoch 85/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2734 - acc: 0.9216
Epoch 86/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2724 - acc: 0.9217
Epoch 87/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2717 - acc: 0.9219
Epoch 88/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2701 - acc: 0.9233
Epoch 89/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2688 - acc: 0.9240
Epoch 90/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.2677 - acc: 0

35000/35000 [==============================] - 0s 2us/step - loss: 0.1642 - acc: 0.9420
Epoch 161/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1608 - acc: 0.9446
Epoch 162/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1590 - acc: 0.9449
Epoch 163/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1577 - acc: 0.9458
Epoch 164/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1568 - acc: 0.9457
Epoch 165/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1556 - acc: 0.9469
Epoch 166/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1550 - acc: 0.9455
Epoch 167/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1542 - acc: 0.9466
Epoch 168/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1536 - acc: 0.9460
Epoch 169/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1525

35000/35000 [==============================] - 0s 2us/step - loss: 0.1163 - acc: 0.9565
Epoch 239/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1165 - acc: 0.9559
Epoch 240/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1159 - acc: 0.9560
Epoch 241/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1154 - acc: 0.9565
Epoch 242/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1158 - acc: 0.9561
Epoch 243/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1161 - acc: 0.9562
Epoch 244/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1152 - acc: 0.9558
Epoch 245/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1144 - acc: 0.9567
Epoch 246/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1146 - acc: 0.9561
Epoch 247/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.1145

...

35000/35000 [==============================] - 0s 2us/step - loss: 0.0585 - acc: 0.9776
Epoch 634/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0585 - acc: 0.9777
Epoch 635/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0584 - acc: 0.9775
Epoch 636/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0592 - acc: 0.9773
Epoch 637/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0587 - acc: 0.9777
Epoch 638/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0587 - acc: 0.9776
Epoch 639/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0588 - acc: 0.9778
Epoch 640/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0581 - acc: 0.9777
Epoch 641/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0578 - acc: 0.9777
Epoch 642/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0584

35000/35000 [==============================] - 0s 2us/step - loss: 0.0523 - acc: 0.9798
Epoch 713/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0528 - acc: 0.9804
Epoch 714/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0525 - acc: 0.9795
Epoch 715/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0527 - acc: 0.9799
Epoch 716/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0542 - acc: 0.9785
Epoch 717/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0517 - acc: 0.9806
Epoch 718/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0522 - acc: 0.9802
Epoch 719/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0528 - acc: 0.9797
Epoch 720/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0523 - acc: 0.9796
Epoch 721/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0518

35000/35000 [==============================] - 0s 2us/step - loss: 0.0481 - acc: 0.9812
Epoch 792/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0484 - acc: 0.9814
Epoch 793/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0475 - acc: 0.9816
Epoch 794/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0474 - acc: 0.9818
Epoch 795/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0481 - acc: 0.9813
Epoch 796/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0468 - acc: 0.9823
Epoch 797/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0467 - acc: 0.9821
Epoch 798/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0469 - acc: 0.9823
Epoch 799/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0470 - acc: 0.9822
Epoch 800/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0473

35000/35000 [==============================] - 0s 2us/step - loss: 0.0427 - acc: 0.9842
Epoch 871/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0427 - acc: 0.9842
Epoch 872/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0428 - acc: 0.9839
Epoch 873/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0426 - acc: 0.9840
Epoch 874/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0426 - acc: 0.9839
Epoch 875/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0421 - acc: 0.9844
Epoch 876/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0421 - acc: 0.9848
Epoch 877/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0426 - acc: 0.9837
Epoch 878/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0426 - acc: 0.9837
Epoch 879/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0429

35000/35000 [==============================] - 0s 2us/step - loss: 0.0394 - acc: 0.9855
Epoch 950/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0389 - acc: 0.9853
Epoch 951/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0387 - acc: 0.9856
Epoch 952/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0396 - acc: 0.9852
Epoch 953/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0391 - acc: 0.9854
Epoch 954/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0386 - acc: 0.9860
Epoch 955/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0399 - acc: 0.9850
Epoch 956/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0392 - acc: 0.9856
Epoch 957/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0388 - acc: 0.9854
Epoch 958/1000
35000/35000 [==============================] - 0s 2us/step - loss: 0.0388

In [22]:
# Save model

classifier.save("pokemon_model.h5")

In [42]:
y_pred = classifier.predict(x_train)
y_pred = (y_pred > 0.5)
train_cm = confusion_matrix(y_pred, y_train)
print("="*20+"  Train Set Evaluation  "+"="*20)
print("Confusion Matrix:")
print(train_cm)
#print("Accuracy on train set: %f" % ((train_cm[0][0]+train_cm[1][1])/(train_cm[0][1]+train_cm[1][0]+train_cm[0][0]+train_cm[1][1])))
print("\nAccuracy: ",accuracy_score(y_pred,y_train))
print("Precision: ",precision_score(y_pred,y_train))
print("Recall: ",recall_score(y_pred,y_train))
print("F1 Score: ",f1_score(y_pred,y_train))
print("\n")
print("="*20+"  Test Set Evaluation  "+"="*20)
print("Confusion Matrix:")
y_test_pred = classifier.predict(x_test)
y_test_pred = (y_test_pred > 0.5)
test_cm = confusion_matrix(y_test_pred,y_test)
print(test_cm)
#print("Accuracy on test set: %f" % ((test_cm[0][0]+test_cm[1][1])/(test_cm[0][1]+test_cm[1][0]+test_cm[0][0]+test_cm[1][1])))
print("\nAccuracy: ",accuracy_score(y_test_pred,y_test))
print("Precision: ",precision_score(y_test_pred,y_test))
print("Recall: ",recall_score(y_test_pred,y_test))
print("F1 Score: ",f1_score(y_test_pred,y_test))


====================  Train Set Evaluation  ====================
Confusion Matrix:
[[18284   204]
 [  225 16287]]

Accuracy:  0.9877428571428571
Precision:  0.9876296161542659
Recall:  0.9863735465116279
F1 Score:  0.9870011817107535


====================  Test Set Evaluation  ====================
Confusion Matrix:
[[7569  377]
 [ 321 6733]]

Accuracy:  0.9534666666666667
Precision:  0.9469760900140647
Recall:  0.9544939041678481
F1 Score:  0.950720135554928
